(advanced:Hcurl)=
# Hcurl and related problems

NGSolve contains a variety of finite elements for apporximating different function spaces:

- `H1`: Lagrange elements approximating functions in $H^1$ 
- `HCurl`: piecewise Nedelec elements approximating functions in $H(curl)$
- `HDiv`: piecewise Brezzi-Douglas-Marini elements approximating functions in $H(div)$ 

One can loop over the basis function and observe that they are build in a hierarchical way.

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *

mesh = Mesh(unit_square.GenerateMesh(maxh=0.5,quad_dominated=True) )

In [ ]:
order = 3
fes = H1(mesh, order=order)
gfu = GridFunction(fes )
gfut = GridFunction(fes , multidim=0)




for i in range(len(gfu.vec)):

    gfu.vec[:] = 0
    gfu.vec[i] = 1

    gfut.AddMultiDimComponent(gfu.vec)

settings = {"camera": {"transformations": [
        {"type": "rotateX", "angle": -45}]}}

Draw(gfut,deformation=True, animate=True ,autoscale = True , min=-0.03, max=0.03, order = 3, settings=settings);

In [ ]:
order = 2
fes = HCurl(mesh, order=order)
gfu = GridFunction(fes)
gfut = GridFunction(fes, multidim=0)


for i in range(len(gfu.vec)):

    gfu.vec[:] = 0
    gfu.vec[i] = 1

    gfut.AddMultiDimComponent(gfu.vec)

Draw(gfut, deformation=False, animate=True, autoscale = True , order = 3,  vectors = { "grid_size":30}, settings=settings);

In [ ]:
order = 2
fes = HDiv(mesh, order=order)
gfu = GridFunction(fes)
gfut = GridFunction(fes, multidim=0)


for i in range(len(gfu.vec)):

    gfu.vec[:] = 0
    gfu.vec[i] = 1

    gfut.AddMultiDimComponent(gfu.vec)

Draw(gfut, deformation=False, animate=True, autoscale = True , order = 3,  vectors = { "grid_size": 30}, settings=settings);

## The eigenvalue problem on the torus:

For simply connected domains the kernel of the curl is given by the image of the gradient. 

In case of a torus the kernel is slightly larger than the image of the gradient.

**problem**: find $u \in H(curl)$ such that
\begin{align*}  
\text{curl} \,u &= 0 \\
u &\neq \nabla \phi \quad \forall \phi \in H^1
\end{align*}

To do so we solve the eigenvalue problem:\
Find $\lambda \in \mathbb{R}$ and $u \in H(curl)$ such that
\begin{align*}
\int_{\Omega} \text{curl} u \cdot \text{curl} v \,dx = \lambda \int_{\Omega} u \cdot v \,dx\\
u \cdot \nabla \phi =0  \quad \forall \phi \in H^1
\end{align*}


Start creating the (double) torus 

In [ ]:
circ = Circle( Pnt(2,0), 1.5).Face()
axis = Axis(Pnt(0,0,0), Dir(0,1,0))#, Dir(1,0,0))
angle = 360
torus = circ.Revolve( axis, angle)
torus2 = torus.Move( (4,0,0), )

geo = torus.Rotate(Axis((0, 0, 0), Y), 180) + torus2


mesh = Mesh(OCCGeometry(geo).GenerateMesh(maxh=3)).Curve(4)


settings = {"camera": {"transformations": [ {"type": "rotateX", "angle": 45}]}}
Draw(mesh, settings=settings);

I will not go too much into detail here, to have a better understanding of the problem I recommend to read the Phd thesis of Sabine Zaglmayr, p 145-150.

In [ ]:
# SetHeapSize(100*1000*1000)

fes = HCurl(mesh, order=2)

print("ndof =", fes.ndof)
u, v = fes.TnT()

a = BilinearForm(curl(u)*curl(v)*dx)
m = BilinearForm(u*v*dx)

apre = BilinearForm(curl(u)*curl(v)*dx + u*v*dx)
pre = Preconditioner(apre, "direct", inverse="sparsecholesky")


The idea is to use the PINVIT (preconditioned inverse iteration) method and let the preconditioner take care of the kernel of the curl.


In [ ]:
with TaskManager():
    a.Assemble()
    m.Assemble()
    apre.Assemble() # when we assemble the BilinearForm , it assembles the matrix as well

    ### Advanced part of the code ###
    
    gradmat, fesh1 = fes.CreateGradient() # create gradient from H1 to HCurl
    gradmattrans = gradmat.CreateTranspose() 
    math1 = gradmattrans @ m.mat @ gradmat   # create projection onto grad H1
    math1[0, 0] += 1     # fix the 1-dim kernel
    invh1 = math1.Inverse(inverse="sparsecholesky") 
    # build the Poisson projector with operator Algebra:
    proj = IdentityMatrix() - gradmat @ invh1 @ gradmattrans @ m.mat 
    projpre = proj @ pre.mat

    evals, evecs = solvers.PINVIT(a.mat, m.mat, pre=projpre, num=12, maxit=20)

In [ ]:
print("Eigenvalues")
for lam in evals:
    print(lam)

In [ ]:

gfu = GridFunction(fes, multidim=len(evecs))
for i in range(len(evecs)):
    gfu.vecs[i].data = evecs[i]
Draw(gfu, mesh, order=2,clipping = {"y":-1, "z":0} ,vectors = { "grid_size": 30},settings=settings);